In [1]:
import pandas as pd
import numpy as np
import openpyxl
from configparser import ConfigParser, ExtendedInterpolation
import os
import sys
from unidecode import unidecode
import requests
from requests.exceptions import RequestException
from datetime import date
from dateutil import parser

from neo4j import GraphDatabase, Query, Result

from neomodel import config, db, install_labels, install_all_labels, remove_all_labels, drop_constraints
from neomodel import (StructuredNode,  RelationshipTo, RelationshipFrom, StructuredRel)
from neomodel import (StringProperty, IntegerProperty, FloatProperty, ArrayProperty, BooleanProperty, DateProperty, UniqueIdProperty, JSONProperty)
# from neomodel import (AliasProperty, RegexProperty, JSONProperty, DateTimeProperty, DateTimeFormatProperty, PointProperty
from neomodel.exceptions import *

from neomodel.integration.pandas import to_dataframe, to_series
from neomodel.integration.numpy import to_ndarray

from socket import gaierror

from pynsee.utils.init_conn import init_conn
from pynsee.macrodata import  get_dataset_list, get_series_list, get_series, get_series_title, search_macrodata, get_dataset
from pynsee.localdata import get_local_metadata, get_local_data
from pynsee.metadata import get_activity_list
from pynsee.sirene import get_dimension_list, search_sirene, get_sirene_data, get_sirene_relatives
from pynsee.geodata import get_geodata_list, get_geodata
from pynsee import get_file_list, download_file

import eurostat 


In [2]:
from modules.config import session

In [3]:
from modules.neomodel_classes import *
from modules.utils import connect, resumeInstance, get_config, getDS
from modules.constructors import addContract, addEnterprise
from modules.location_management import getDeliveryLocation, getLocationCode

In [4]:
try:
    driver = connect(session, 'self-managed')
except gaierror:
    pass

fichier de configuration C:\Users\Utilisateur\projets\neo4j\config.ini


In [ ]:
session

In [ ]:
remove_all_labels()
install_all_labels()



In [5]:

types_decp = {'titulaire_id_1': str, 'titulaire_id_2': str, 'titulaire_id_3': str, 'idAccordCadre': str, 'ccag': str,
              'typeGroupementOperateurs': str, 'technique': str, 'modaliteExecution': str, 'lieuExecution.code': str,
              'lieuExecution.nom': str, 'TypePrix': str, 'typePrix': str, 'typesPrix': str}
decp = pd.read_csv(session['dataset'], sep= ';', dtype= types_decp)

# decp['lieuExecution.code'].fillna('', inplace= True)
decp.fillna({'lieuExecution.code': ''}, inplace= True)
# suppression des identifiants non SIRET (TVA, hors UE etc)
decp = decp.loc[(decp['titulaire_typeIdentifiant_1'] == 'SIRET') & 
                ((decp['titulaire_typeIdentifiant_2'].isna())|(decp['titulaire_typeIdentifiant_2'] == 'SIRET')) &
                ((decp['titulaire_typeIdentifiant_3'].isna())|(decp['titulaire_typeIdentifiant_3'] == 'SIRET'))]

decp.loc[decp.titulaire_denominationSociale_1.isna(), 'titulaire_denominationSociale_1'] = \
decp[decp.titulaire_denominationSociale_1.isna()].titulaire_id_1.apply(lambda x: getDS(str(x)[0:14]))

decp.loc[decp.titulaire_typeIdentifiant_2.notna() & decp.titulaire_denominationSociale_2.isna(), 'titulaire_denominationSociale_2'] = \
decp[decp.titulaire_typeIdentifiant_2.notna() & decp.titulaire_denominationSociale_2.isna()].titulaire_id_2.apply(lambda x: getDS(str(x)[0:14]))

decp.loc[decp.titulaire_typeIdentifiant_3.notna() & decp.titulaire_denominationSociale_3.isna(), 'titulaire_denominationSociale_3'] = \
decp[decp.titulaire_typeIdentifiant_3.notna() & decp.titulaire_denominationSociale_3.isna()].titulaire_id_3.apply(lambda x: getDS(str(x)[0:14]))


This function may return personal data, please check and comply with the legal framework relating to personal data protection !
API query number limit reached - function might be slowed down


In [8]:
decp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192773 entries, 0 to 193609
Data columns (total 54 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   titulaire_denominationSociale_1     192773 non-null  object 
 1   titulaire_id_1                      192773 non-null  object 
 2   titulaire_typeIdentifiant_1         192773 non-null  object 
 3   titulaire_denominationSociale_2     16267 non-null   object 
 4   titulaire_id_2                      16264 non-null   object 
 5   titulaire_typeIdentifiant_2         16265 non-null   object 
 6   titulaire_denominationSociale_3     8366 non-null    object 
 7   titulaire_id_3                      8364 non-null    object 
 8   titulaire_typeIdentifiant_3         8366 non-null    object 
 9   procedure                           187122 non-null  object 
 10  nature                              192580 non-null  object 
 11  codeCPV                        

In [ ]:
decp[0:].apply(addContract, axis= 1)

XXXXXXXXXX type non trouvé XXXXXXXX collectivite doutre-mer a statut particulier FR-NC


In [13]:
siret = '43015156300011'
siret = '39930737000011'
df = search_sirene(variable = ['siret'], number= 1,
                   pattern = [siret], phonetic_search = False, legal=True, closed=True)
df

,siren,nic,siret,dateDebut,dateCreationEtablissement,dateCreationUniteLegale,dateFin,denominationUniteLegale,nomUniteLegale,prenomUsuelUniteLegale,...,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement
0,399307370,00011,39930737000011,2018-01-01,1995-01-01,1995-01-01,None,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
1,399307370,00011,39930737000011,2010-01-01,1995-01-01,1995-01-01,2017-12-31,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
2,399307370,00011,39930737000011,2009-05-01,1995-01-01,1995-01-01,2009-12-31,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
3,399307370,00011,39930737000011,2008-01-01,1995-01-01,1995-01-01,2009-04-30,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
4,399307370,00011,39930737000011,2006-05-15,1995-01-01,1995-01-01,2007-12-31,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
5,399307370,00011,39930737000011,2003-12-25,1995-01-01,1995-01-01,2006-05-14,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
6,399307370,00011,39930737000011,1996-12-25,1995-01-01,1995-01-01,2003-12-24,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None
7,399307370,00011,39930737000011,1995-01-01,1995-01-01,1995-01-01,1996-12-24,EIFFAGE ROUTE SUD OUEST,None,None,...,None,None,None,None,None,None,None,None,None,None


In [19]:
decp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193610 entries, 0 to 193609
Data columns (total 54 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   titulaire_denominationSociale_1     193610 non-null  object 
 1   titulaire_id_1                      192935 non-null  object 
 2   titulaire_typeIdentifiant_1         193610 non-null  object 
 3   titulaire_denominationSociale_2     16421 non-null   object 
 4   titulaire_id_2                      16418 non-null   object 
 5   titulaire_typeIdentifiant_2         16419 non-null   object 
 6   titulaire_denominationSociale_3     8468 non-null    object 
 7   titulaire_id_3                      8430 non-null    object 
 8   titulaire_typeIdentifiant_3         8468 non-null    object 
 9   procedure                           187889 non-null  object 
 10  nature                              193417 non-null  object 
 11  codeCPV                   

In [20]:
decp[14300:14320]['lieuExecution.code']

14300    13138
14301       90
14302    09318
14303    57050
14304    88320
14305    09113
14306    97200
14307    31600
14308    38610
14309    91000
14310      974
14311    31300
14312    97400
14313         
14314    31220
14315    69123
14316    83390
14317    59132
14318    13530
14319    69123
Name: lieuExecution.code, dtype: object

In [ ]:
df = search_sirene(variable = ['siret'], number= 1,
                   pattern = [siret], phonetic_search = False, legal=True, closed=True)


In [ ]:
sd = get_dimension_list()

In [ ]:
sd.loc[0:20, 'siret_columns']

In [ ]:
driver.close()

In [ ]:
request = "call db.labels()"
# request = "match (n) return DISTINCT labels(n) AS labs"

result = driver.execute_query(request, result_transformer_= Result.to_df)
result.loc[1, 'label']

In [ ]:
request = """WITH \"MATCH (c:Contract) RETURN c\" as query
CALL apoc.export.csv.query(query, NULL, {stream: true})
YIELD data
RETURN data;"""

result = driver.execute_query(request)
result

In [ ]:
result.records